In [1]:
import pandas as pd

In [19]:
df = pd.read_csv("all_materials_combined.csv")

In [20]:
df.loc[:1]

,Material,Yield strength (MPa),Tensile strength (MPa),Elongation at break (%),Hardness,Modulus of elasticity (GPa),Tensilemodulus(MPa),Tensilestrength (MPa),Elongationat break (%),Flexuralstrength(MPa),Flexuralmodulus (GPa)
0,Aluminium 2007 / 3.1645 / Al-CuMgPb,210 – 250,370,6 – 8,130,72.5,NaN,NaN,NaN,NaN,NaN
1,Aluminium 2017A / 3.1325 / Al-Cu4Mg,135 – 240,250 – 370,8 – 12,45 – 105,72.5,NaN,NaN,NaN,NaN,NaN


In [17]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import lxml


def fetch_tables_to_csv(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the webpage: {e}")
        return

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all tables in the HTML
    tables = soup.find_all('table')
    # Prepare to store all tables
    all_materials = []

    for idx, table in enumerate(tables):
        rows = []
        
        # Find header cells
        header_row = table.find('tr')
        if not header_row:
            continue
        headers = [th.get_text(strip=True) for th in header_row.find_all(['td', 'th'])]
        
        # Standardize first header name
        if headers:
            headers[0] = 'Material'  # no matter if it was 'Alloy' or 'Material', rename it

        
        
        # Iterate over all rows after the header
        for row in table.find_all('tr')[1:]:
            cols = row.find_all('td')
            cols = [ele.get_text(strip=True) for ele in cols]
            if cols:
                rows.append(cols)
        
        # Convert to DataFrame
        if rows:
            df = pd.DataFrame(rows, columns=headers)
            all_materials.append(df)
    
    
    # Option 2: Combine all into one big CSV (optional)
    combined_df = pd.concat(all_materials, ignore_index=True)
    combined_df.to_csv('all_materials_combined.csv', index=False)

# URL of the webpage containing the tables
url = "https://xometry.pro/en/articles/cnc-machining-materials/"

# Call the function to fetch tables and save as CSV
fetch_tables_to_csv(url)


In [10]:
!pip install lxml


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 18.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [18]:
combined_df

NameError: name 'combined_df' is not defined

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fetch the webpage
url = "https://xometry.pro/en/articles/cnc-machining-materials/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Find all tables
tables = soup.find_all('table')

# Define header mapping
header_mapping = {
    'Alloy': 'Material',
    'Material': 'Material',
    'Tensile strength (MPa)': 'Tensile strength (MPa)',
    'Tensilestrength (MPa)': 'Tensile strength (MPa)',
    'Tensile Strength (MPa)': 'Tensile strength (MPa)',
    'Tensilestrength': 'Tensile strength (MPa)',
    'Elongation at break (%)': 'Elongation at break (%)',
    'Elongation (%)': 'Elongation at break (%)',
    'Elongation': 'Elongation at break (%)',
    'Elongationat break (%)': 'Elongation at break (%)',
    'Yield strength (MPa)': 'Yield strength (MPa)',
    'Hardness': 'Hardness',
    'Modulus of elasticity (GPa)': 'Modulus of elasticity (GPa)',
    'Tensile modulus (MPa)': 'Tensile modulus (MPa)',
    'Tensilemodulus(MPa)': 'Tensile modulus (MPa)',
    'Flexural strength (MPa)': 'Flexural strength (MPa)',
    'Flexuralstrength(MPa)': 'Flexural strength (MPa)',
    'Flexural modulus (GPa)': 'Flexural modulus (GPa)',
    'Flexuralmodulus (GPa)': 'Flexural modulus (GPa)',
}

# Prepare storage
all_materials = []

for idx, table in enumerate(tables):
    rows = []
    header_row = table.find('tr')
    if not header_row:
        continue
    headers = [th.get_text(strip=True) for th in header_row.find_all(['td', 'th'])]
    headers = [header_mapping.get(h, h) for h in headers]

    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        cols = [ele.get_text(strip=True) for ele in cols]
        if cols:
            rows.append(cols)

    if rows:
        df = pd.DataFrame(rows, columns=headers)
        all_materials.append(df)

# Combine
combined_df = pd.concat(all_materials, ignore_index=True)

# --- Post-processing to merge similar columns ---

# Fill missing standard columns from alternates
if 'Tensile strength (MPa)' not in combined_df.columns and 'Tensilestrength (MPa)' in combined_df.columns:
    combined_df['Tensile strength (MPa)'] = combined_df['Tensilestrength (MPa)']

if 'Tensile strength (MPa)' in combined_df.columns and 'Tensilestrength (MPa)' in combined_df.columns:
    combined_df['Tensile strength (MPa)'] = combined_df['Tensile strength (MPa)'].fillna(combined_df['Tensilestrength (MPa)'])

if 'Elongation at break (%)' not in combined_df.columns and 'Elongation break (%)' in combined_df.columns:
    combined_df['Elongation at break (%)'] = combined_df['Elongation break (%)']

if 'Elongation at break (%)' in combined_df.columns and 'Elongation break (%)' in combined_df.columns:
    combined_df['Elongation at break (%)'] = combined_df['Elongation at break (%)'].fillna(combined_df['Elongationat break (%)'])

if 'Flexural strength (MPa)' not in combined_df.columns and 'Flexuralstrength(MPa)' in combined_df.columns:
    combined_df['Flexural strength (MPa)'] = combined_df['Flexuralstrength(MPa)']

if 'Flexural strength (MPa)' in combined_df.columns and 'Flexuralstrength(MPa)' in combined_df.columns:
    combined_df['Flexural strength (MPa)'] = combined_df['Flexural strength (MPa)'].fillna(combined_df['Flexuralstrength(MPa)'])

if 'Flexural modulus (GPa)' not in combined_df.columns and 'Flexuralmodulus (GPa)' in combined_df.columns:
    combined_df['Flexural modulus (GPa)'] = combined_df['Flexuralmodulus (GPa)']

if 'Flexural modulus (GPa)' in combined_df.columns and 'Flexuralmodulus (GPa)' in combined_df.columns:
    combined_df['Flexural modulus (GPa)'] = combined_df['Flexural modulus (GPa)'].fillna(combined_df['Flexuralmodulus (GPa)'])

# Drop the messy alternate columns now
columns_to_drop = ['Tensilestrength (MPa)', 'Tensilestrength', 'Elongationat break (%)', 'Flexuralstrength(MPa)', 'Flexuralmodulus (GPa)', 'Tensilemodulus(MPa)']
combined_df = combined_df.drop(columns=[col for col in columns_to_drop if col in combined_df.columns])

# Save final clean CSV
combined_df.to_csv('all_materials_cleaned.csv', index=False)

print("✅ Materials extracted, merged cleanly, and saved to 'all_materials_cleaned.csv'.")


✅ Materials extracted, merged cleanly, and saved to 'all_materials_cleaned.csv'.
